In [1]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt

# DP Formula
# V[x] = max(0, [xp + (1-x)q] V(x_win) + [x(1-p) + (1-x)(1-q)] V(x_lose) + [xp + (1-x)q] - c ))

In [2]:
roundfactor = 4

def update_prior(x,p,q,c,result):
    if result == True:
        out = p*x/(win(x,p,q))
    else:
        out = (1 - p)*x/(lose(x,p,q))
    return round(out,roundfactor)

def expected_outcome(x,p,q,c):
    out = win(x,p,q) - c
    return round(out,roundfactor)

def win(x,p,q):
    return x*p + (1-x)*q

def lose(x,p,q):
    return x*(1-p) + (1-x)*(1-q)